# OCT 1, 2025 - Tinkering with Data Analysis

#### Some ideas:
* response time (b/w each "category" of petitions, individuals vs big companies,   
* count of big companies vs individuals
* type of requests being made
* success rate: type of requests being denied vs approved
* what specifically gets a petition denied/approved?

In [1]:
%pip install seaborn
%pip install pandas
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
petitions_2017 = pd.read_csv("data/2017_PETITIONS_COMPLETE_071825_MP.xlsx - Sheet1.csv")
responses_2017 = pd.read_csv("data/2017_RESPONSES_COMPLETE_071825_MP.xlsx - Sheet1.csv")
pharma_comps = pd.read_csv("data/WR-Issued-June-2024-Approved-Active-Moiety-6-27-24.csv")

In [4]:
petitions_2017.sort_values("Identity of Submitting Entity").head(3)

File Name Date of Petition  \
34   FDA-2017-P-1251-0001_Citizen_Petition_from_Ame...        2/27/2017   
138  FDA-2017-P-4372-0002_Citizen_Petition_from_AEG...              NaN   
52   FDA-2017-P-1990-0001_Citizen_Petition_from_AID...        3/23/2017   

                                         Date Comments  \
34                                                 NaN   
138  No date recognized by ChatGPT nor apparent thr...   
52   Letterhead dated March 23, but stamp reflects ...   

                         Identity of Submitting Entity Representation Details  \
34   - American Feed Industry Association (AFIA)\n-...         Not mentioned.   
138                             AEGIS Regulatory, Inc.         Not mentioned.   
52                                        AIDP, Inc.\n         Not mentioned.   

                         Cited Statutes or Regulations  \
34   - 21 U.S.C. §§ 351(a)(2)(B), 360b(m), 371(a), ...   
138  - 21 C.F.R. § 10.30 (Citizen Petition)\n- 21 C...   
52   - 21 C.F.R. § 10.30\n- 21 C.F.R. § 101.9(c)(6)...   

                               FDA Action Commented On  \
34   The application of 21 C.F.R. Part 11 (electron...   
138  Enforcement and importation practices regardin...   
52   Definition of dietary fiber under the revised ...   

                                      Requested Action  \
34   - Exempt all medicated feed cGMP-related recor...   
138  - Prohibit the entry of adulterated cosmetic p...   
52   Amend 21 C.F.R. § 101.9(c)(6)(i) to include xy...   

                             Justification for Request  Status of Review  \
34   - High cost and burden of Part 11 compliance (...             False   
138  - Reports of adverse health effects including ...             False   
52   AIDP presented evidence that XOS provides mult...             False   

    Review Comments  
34              NaN  
138             NaN  
52              NaN

In [5]:
responses_2017.head(3)

File Name Date of Response  \
0  FDA-2017-P-0052-0150_Interim_Response_from_CBE...              NaN   
1  FDA-2017-P-0052-0203_Response_letter_from_FDA_...              NaN   
2  FDA-2017-P-0115-0007_Memorandum_of_suitability...        6/12/2023   

                                       Date Comments  \
0  No date recognized by ChatGPT, but found in si...   
1  No date recognized by ChatGPT, but found in si...   
2                                                NaN   

                               Responding FDA Center Response to Petition  \
0  Center for Biologics Evaluation and Research (...     Interim Response   
1  Center for Biologics Evaluation and Research (...               Denied   
2    Center for Drug Evaluation and Research (CDER)             Withdrawn   

                       Cited Statutes or Regulations  \
0  21 CFR 10.30(e)(2) (FDA regulations on citizen...   
1  - Section 361 of the Public Health Service Act...   
2                                   Not applicable.    

                          Justification for Response  Status of Review  \
0  The FDA cites "existence of other FDA prioriti...             False   
1  The FDA denied the petition based on the follo...             False   
2  - FDA contacted the petitioner (Jubilant Gener...             False   

  Review Comments  
0             NaN  
1             NaN  
2             NaN

In [6]:
pharma_comps.head()

Active Moiety                       Sponsor Date Issued  \
0                          Abacavir          Glaxo Wellcome, Inc.     8/20/98   
1  Abacavir/Dolutegravir/Lamivudine       ViiV Healthcare Company     5/16/18   
2                         Abatacept  Bristol-Myers Squibb Company     9/13/13   
3                       Abemaciclib               Eli Lilly & Co.      6/7/19   
4                       Acamprosate                   Forest Labs     11/9/05   

   Unnamed: 3  Unnamed: 4  
0         NaN         NaN  
1         NaN         NaN  
2         NaN         NaN  
3         NaN         NaN  
4         NaN         NaN

## Feature Engineering

### Labeling the submitters

#### Cleaning "Identity of Submitting Entity"
* petitions filed by individuals usually labeled by:
    * "Individual citizen petition..."
    * "Multiple co-signatories:..."
    * "Public Citizen" but be careful with "Public Citizen’s Health Research Group"
    * specific names first and last (but whole title not ending in LLP, LLC, Inc, P.C; also be careful with companies having "normal" names at the beginning like wilson, kleinfeld, but dont end in llp, llc etc)
    * "Anonymous (redacted)" (?)
      
    * ***individual: 'individual citizen', 'multiple co-signatories', 'anonymous', ' md', ' jd', ' pharmd', ' ph.d'*** 

* petitions filed by corps usually labeled by:
    * ***industry/corporate: 'llc', 'inc', 'pharma', 'ltd', limited', 'corp', 'laboratories', 'technologies'***

    * ***law/consulting: 'llp', 'p.c.', 'law', 'legal', 'consultant', 'regulatory', 'associates'***

    * ***advocacy/academic: 'association', 'university', 'center for', 'society', 'foundation'***


#### What to clean
* "\n", "\t" at the end of some
* different variations
  * 'Kleinfeld, Kaplan & Becker LLP',
  * 'Kleinfeld, Kaplan & Becker, LLP',
  * 'Kleinfeld, Kaplan and Becker LLP\t',
  * 'Kleinfeld, Kaplan and Becker, LLP\n'

  * 'Apotex Inc.\n',
  * 'Apotex Inc. and Apobiologix (a Division of Apotex Inc.) \n'
    
  * 'INC Research, LLC',
  * 'INC Research/inVentiv Health'
* multiple corporations in single cell separated by dashes - or "and"
  * '- American Feed Industry Association (AFIA)\n- National Grain and Feed Association (NGFA)'
  * 'Abhai, LLC and KVK-Tech, Inc.'
* edge cases:
  * 'Kent Heckenlively, JD; also signed by Rima E. Laibow, MD and Ralph Fucetola, JD on behalf of Natural Solutions Foundation and Institute for Health Research respectively \n'
  * 'NJOY, along with:\n- Vapor Technology Association\n- SFATA\n- National Association of Tobacco Outlets, Inc.\n- CITMA\n- Turning Point Brands, Inc.\n- Mistic Electronic Cigarettes\n- Johnson Creek Vapor Company\n- Nicopure Labs LLC\n- Five Pawns Inc.\n- Gaiatrend USA\n- Glas, LLC\n- Shift Ventures, Inc.\n- E-Alternative Solutions, Inc.\n- Saffire Vapor'

In [7]:
#petitions_2017["Identity of Submitting Entity"].sort_values().unique()

In [8]:
# cleaning "Identity of Submitting Entity"
petitions_2017_mod = petitions_2017.copy()

# remove rows w/ no submitting entity
petitions_2017_mod = petitions_2017_mod.dropna(subset = ["Identity of Submitting Entity"])

# removing "\n", "\t", "\n-", "\n•\u202f"
petitions_2017_mod["Submitter"] = petitions_2017_mod["Identity of Submitting Entity"].astype(str)
petitions_2017_mod["Submitter"] = petitions_2017_mod["Submitter"].str.strip().str.replace("\n-", " ").str.replace("\n•\u202f", " ").str.replace("\u202f", " ")

#petitions_2017["Submitter"].sort_values().unique()

In [9]:
# cleaning pharmaceutical companies corpus

# drop last row bc NaN
pharma_comps = pharma_comps.drop(len(pharma_comps) - 1)

# remove inc, lp, etc
def normalize_company_name(name):
    name_lower = name.lower()
    
    # looks for common suffixes (inc, llc, etc.) and removes them from the end of the comp name
    name_lower = re.sub(r'(\s*,\s*|\s+)(inc|llc|l\.p|lp|limited|corp|co|ltd)\.?\s*$', '', name_lower)
    
    # remove any remaining punctuation and extra whitespace
    name_lower = re.sub(r'[^\w\s]', '', name_lower)
    name_lower = name_lower.strip()
    
    return name_lower
    
pharma_names_list = pharma_comps["Sponsor"].str.lower().str.strip().sort_values().unique()
#display(pharma_names_list)

pharma_names_norm = pharma_comps["Sponsor"].apply(normalize_company_name).sort_values().unique()
#display(pharma_names_norm.head)

In [10]:
#if any(word in 'some one long two phrase three' for word in list_):
# list of pharma companies for categorizing https://share.google/Lzg1nML50MNBjpQkP

# categorize each submitter
individual_terms = ['individual', 'citizen', 'individual citizen', 'multiple co-signatories', ' md', ' jd', ' pharmd', ' ph.d', ' senator']
indus_corp_terms = ['llc', 'inc', 'pharma', 'ltd', 'limited', 'corp', 'laboratories', 'technologies']
outlier_indus = ['unilever']
law_consult_terms = ['llp', 'p.c.', 'p.c', 'law', 'legal', 'consultant', 'consulting', 'regulatory', 'associates', 'solution']
advoc_academic_terms = ['association', 'university', 'center for', 'society', 'foundation', 'research group', 'research', 
                        'lab', 'registry', 'institute', 'council', 'choice']

def categorize_submitter(submitter_title):
    submitter_title_lower = submitter_title.lower()
    #submitter_title_norm = normalize_company_name(submitter_title)
    
    if pd.isna(submitter_title) or "anonymous" in submitter_title_lower:
        return "other"
        
    elif submitter_title_lower in pharma_names_list or submitter_title_lower in pharma_names_norm:
        return "industry/corporation"
        
    # for each identifier word in individual, industry, etc, check if that word exists in the title
    if submitter_title_lower in outlier_indus or any(word in submitter_title_lower for word in indus_corp_terms):
        return "industry/corporation"
        
    elif any(word in submitter_title_lower for word in law_consult_terms):
        return "law/consulting"
        
    elif any(word in submitter_title_lower for word in advoc_academic_terms):
        return "advocacy/academic"
        
    elif any(word in submitter_title_lower for word in individual_terms):
        return "individual"

    # check for short individual's names (usually first, middle, last)
    elif len(submitter_title_lower.split()) <= 3:
        return "individual"

    else:
        return "other"

In [11]:
petitions_2017_mod["Submitter Type"] = petitions_2017_mod["Submitter"].apply(categorize_submitter)
#petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "law/consulting"]

petitions_2017_mod["Submitter Type"].value_counts()

Submitter Type
industry/corporation    128
law/consulting           78
advocacy/academic        15
individual               13
other                     2
Name: count, dtype: int64

In [12]:
display(petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "advocacy/academic"][["Submitter", "Submitter Type"]])
display(petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "other"][["Submitter", "Submitter Type"]])
display(petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "individual"][["Submitter", "Submitter Type"]])

Submitter     Submitter Type
0                               Donor Sibling Registry  advocacy/academic
5    Malaysian Rubber Export Promotion Council (MREPC)  advocacy/academic
11   Union of Concerned Scientists, Center for Scie...  advocacy/academic
34   - American Feed Industry Association (AFIA)  N...  advocacy/academic
38                       The Good Food Institute (GFI)  advocacy/academic
66                         Consumers for Dental Choice  advocacy/academic
75           The Tatia Oden French Memorial Foundation  advocacy/academic
103                                        Salter Labs  advocacy/academic
146                  American Dental Association (ADA)  advocacy/academic
152                   Wilson Disease Association (WDA)  advocacy/academic
165  Multiple co-signatories:  Andrew Kolodny, M.D....  advocacy/academic
170        Post-Finasteride Syndrome Foundation (PFSF)  advocacy/academic
221             Public Citizen’s Health Research Group  advocacy/academic
227              St. Jude Children’s Research Hospital  advocacy/academic
230  Natural Products Association (NPA) and Bergstr...  advocacy/academic

Submitter Submitter Type
98   Anonymous (redacted)          other
100  Anonymous (redacted)          other

Submitter Submitter Type
21                                      Public Citizen     individual
22                                          David Wong     individual
69   Senator Charles E. Schumer and Senator Kirsten...     individual
92   Individual citizen petition submitted jointly ...     individual
93   Philip Spiller, along with:  George A. Gellert...     individual
102                                      Michael Boggi     individual
133                                  William Bonificio     individual
151                                Chul-Hi Park, Ph.D.     individual
163                                 Dennis L. Ryll, MD     individual
164                                 Dennis L. Ryll, MD     individual
205                                    Diana Wierzchos     individual
209                                     Public Citizen     individual
213          Dr. S. Albert Edwards, PharmD, RAC, FRAPS     individual

In [13]:
# download to check bc too many rows
filter_industry = petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "industry/corporation"][["Submitter", "Submitter Type"]]
filter_industry.to_csv("industry_check.csv", index=False)

filter_law = petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "law/consulting"][["Submitter", "Submitter Type"]]
filter_law.to_csv("law_check.csv", index=False)

In [14]:
petitions_2017_mod.head(3)

File Name Date of Petition  \
0  FDA-2017-P-0052-0001_Citizen_Petition_from_Don...         1/1/2017   
1  FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...       12/22/2016   
2  FDA-2017-P-0137-0001_Citizen_Petition_from_Mac...         1/9/2017   

  Date Comments     Identity of Submitting Entity Representation Details  \
0           NaN          Donor Sibling Registry\n         Not mentioned.   
1           NaN         Jubilant Generics Limited         Not mentioned.   
2           NaN  Macleods Pharmaceuticals Limited         Not mentioned.   

                       Cited Statutes or Regulations  \
0  - 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...   
1  - Section 505(j)(2)(C) of the FD&C Act [21 U.S...   
2  - Section 505(j) of the Federal Food, Drug, an...   

                             FDA Action Commented On  \
0  Lack of regulation and oversight in the sperm ...   
1  FDA’s requirement to approve an ANDA petition ...   
2  The petition comments on the absence of a curr...   

                                    Requested Action  \
0  The petitioner asks the FDA Commissioner to in...   
1  FDA is requested to determine that Levetiracet...   
2  Macleods requests FDA to designate the approve...   

                           Justification for Request  Status of Review  \
0  The petition outlines that the U.S. cryobank i...             False   
1  - The proposed product matches the RLD in conc...             False   
2  - The current RLD (Pyrazinamide 500 mg by Dava...             False   

  Review Comments                         Submitter        Submitter Type  
0             NaN            Donor Sibling Registry     advocacy/academic  
1             NaN         Jubilant Generics Limited  industry/corporation  
2             NaN  Macleods Pharmaceuticals Limited  industry/corporation

### Combining petitions and responses data 

In [15]:
responses_2017.head(3)

File Name Date of Response  \
0  FDA-2017-P-0052-0150_Interim_Response_from_CBE...              NaN   
1  FDA-2017-P-0052-0203_Response_letter_from_FDA_...              NaN   
2  FDA-2017-P-0115-0007_Memorandum_of_suitability...        6/12/2023   

                                       Date Comments  \
0  No date recognized by ChatGPT, but found in si...   
1  No date recognized by ChatGPT, but found in si...   
2                                                NaN   

                               Responding FDA Center Response to Petition  \
0  Center for Biologics Evaluation and Research (...     Interim Response   
1  Center for Biologics Evaluation and Research (...               Denied   
2    Center for Drug Evaluation and Research (CDER)             Withdrawn   

                       Cited Statutes or Regulations  \
0  21 CFR 10.30(e)(2) (FDA regulations on citizen...   
1  - Section 361 of the Public Health Service Act...   
2                                   Not applicable.    

                          Justification for Response  Status of Review  \
0  The FDA cites "existence of other FDA prioriti...             False   
1  The FDA denied the petition based on the follo...             False   
2  - FDA contacted the petitioner (Jubilant Gener...             False   

  Review Comments  
0             NaN  
1             NaN  
2             NaN

In [16]:
responses_2017_mod = responses_2017.drop(["Status of Review", "Review Comments"], axis = "columns")
#responses_2017_mod

### Making unique ids, labeling document type (og petit, reconsideration), boolean column for if petition was withdrawn

In [30]:
# use numbers after "FDA-2017-P-..." as unique id
petitions_2017_mod["File ID"] = petitions_2017_mod["File Name"].str.extract(r'(FDA-\d{4}-P-\d{4})')
petitions_2017_mod["File ID"] = petitions_2017_mod["File ID"].str.lower()
display(petitions_2017_mod.head(3))

responses_2017_mod["File ID"] = responses_2017_mod["File Name"].str.extract(r'(FDA-\d{4}-P-\d{4})')
responses_2017_mod["File ID"] = responses_2017_mod["File ID"].str.lower()
display(responses_2017_mod.head(3))

File Name Date of Petition  \
0  FDA-2017-P-0052-0001_Citizen_Petition_from_Don...         1/1/2017   
1  FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...       12/22/2016   
2  FDA-2017-P-0137-0001_Citizen_Petition_from_Mac...         1/9/2017   

  Date Comments     Identity of Submitting Entity Representation Details  \
0           NaN          Donor Sibling Registry\n         Not mentioned.   
1           NaN         Jubilant Generics Limited         Not mentioned.   
2           NaN  Macleods Pharmaceuticals Limited         Not mentioned.   

                       Cited Statutes or Regulations  \
0  - 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...   
1  - Section 505(j)(2)(C) of the FD&C Act [21 U.S...   
2  - Section 505(j) of the Federal Food, Drug, an...   

                             FDA Action Commented On  \
0  Lack of regulation and oversight in the sperm ...   
1  FDA’s requirement to approve an ANDA petition ...   
2  The petition comments on the absence of a curr...   

                                    Requested Action  \
0  The petitioner asks the FDA Commissioner to in...   
1  FDA is requested to determine that Levetiracet...   
2  Macleods requests FDA to designate the approve...   

                           Justification for Request  Status of Review  \
0  The petition outlines that the U.S. cryobank i...             False   
1  - The proposed product matches the RLD in conc...             False   
2  - The current RLD (Pyrazinamide 500 mg by Dava...             False   

  Review Comments                         Submitter        Submitter Type  \
0             NaN            Donor Sibling Registry     advocacy/academic   
1             NaN         Jubilant Generics Limited  industry/corporation   
2             NaN  Macleods Pharmaceuticals Limited  industry/corporation   

           File ID  Was_Withdrawn      Document Type  Withdrawal File  
0  fda-2017-p-0052          False  original petition            False  
1  fda-2017-p-0115          False  original petition            False  
2  fda-2017-p-0137           True  original petition            False

File Name Date of Response  \
0  FDA-2017-P-0052-0150_Interim_Response_from_CBE...              NaN   
1  FDA-2017-P-0052-0203_Response_letter_from_FDA_...              NaN   
2  FDA-2017-P-0115-0007_Memorandum_of_suitability...        6/12/2023   

                                       Date Comments  \
0  No date recognized by ChatGPT, but found in si...   
1  No date recognized by ChatGPT, but found in si...   
2                                                NaN   

                               Responding FDA Center Response to Petition  \
0  Center for Biologics Evaluation and Research (...     Interim Response   
1  Center for Biologics Evaluation and Research (...               Denied   
2    Center for Drug Evaluation and Research (CDER)             Withdrawn   

                       Cited Statutes or Regulations  \
0  21 CFR 10.30(e)(2) (FDA regulations on citizen...   
1  - Section 361 of the Public Health Service Act...   
2                                   Not applicable.    

                          Justification for Response          File ID  
0  The FDA cites "existence of other FDA prioriti...  fda-2017-p-0052  
1  The FDA denied the petition based on the follo...  fda-2017-p-0052  
2  - FDA contacted the petitioner (Jubilant Gener...  fda-2017-p-0115

In [47]:
# FINALIZING petitions data
# adding withdrawals as a feature (separate column)
def withdrawn_petit(df):
    df["Withdrawal File"] = df["File Name"].str.lower().str.contains("withdraw")
    withdrawals_id = df[df["Withdrawal File"] == True]["File ID"]
    df["Was_Withdrawn"] = df["File ID"].isin(withdrawals_id)
    #df.drop("Withdrawal File", axis = "columns", inplace = True)

# OCT 15 REVISED: a column for document type (reconsideration, og petition)
def get_doc_type(row):
    file_name_lower = str(row["File Name"]).lower()
    
    if "reconsideration" in file_name_lower:
        return "reconsideration"
        
    return "original petition"

withdrawn_petit(petitions_2017_mod)
petitions_2017_mod["Document Type"] = petitions_2017_mod.apply(get_doc_type, axis = 1)

# cleaning
petitions_2017_main = petitions_2017_mod[petitions_2017_mod["Withdrawal File"] == False].copy()
petitions_2017_main = petitions_2017_main.drop(["Identity of Submitting Entity", "Withdrawal File", "Status of Review", 
                                                "Date Comments", "Representation Details", "Review Comments"], axis = "columns")
petitions_2017_main = petitions_2017_main.rename(columns = {"Cited Statutes or Regulations": "Petition Cited Statutes or Regulations", "File Name": "Petition File Name"})
new_column_order = ['File ID', 'Petition File Name', 'Date of Petition', 'Was_Withdrawn', 'Document Type', 'Submitter', 'Submitter Type',  
                    'Petition Cited Statutes or Regulations', 'FDA Action Commented On', 'Requested Action', 'Justification for Request']

petitions_2017_main = petitions_2017_main[new_column_order]

petitions_2017_main.to_csv("petitions_2017_main.csv", index=False)
petitions_2017_main.head()

File ID                                 Petition File Name  \
0  fda-2017-p-0052  FDA-2017-P-0052-0001_Citizen_Petition_from_Don...   
1  fda-2017-p-0115  FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...   
2  fda-2017-p-0137  FDA-2017-P-0137-0001_Citizen_Petition_from_Mac...   
4  fda-2017-p-0176  FDA-2017-P-0176-0001_Citizen_Petition_from_Ame...   
5  fda-2017-p-0290  FDA-2017-P-0290-0001_Citizen_Petition_from_Mal...   

  Date of Petition  Was_Withdrawn      Document Type  \
0         1/1/2017          False  original petition   
1       12/22/2016          False  original petition   
2         1/9/2017           True  original petition   
4         1/9/2017          False  original petition   
5        1/17/2017          False  original petition   

                                           Submitter        Submitter Type  \
0                             Donor Sibling Registry     advocacy/academic   
1                          Jubilant Generics Limited  industry/corporation   
2                   Macleods Pharmaceuticals Limited  industry/corporation   
4                  Kleinfeld, Kaplan and Becker, LLP        law/consulting   
5  Malaysian Rubber Export Promotion Council (MREPC)     advocacy/academic   

              Petition Cited Statutes or Regulations  \
0  - 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...   
1  - Section 505(j)(2)(C) of the FD&C Act [21 U.S...   
2  - Section 505(j) of the Federal Food, Drug, an...   
4  - 21 C.F.R. § 10.30 (Citizen Petition procedur...   
5  - FDA Docket No. FDA-2015-N-5017 (Final Rule)\...   

                             FDA Action Commented On  \
0  Lack of regulation and oversight in the sperm ...   
1  FDA’s requirement to approve an ANDA petition ...   
2  The petition comments on the absence of a curr...   
4  The FDA’s proposed amendment to 21 C.F.R. § 10...   
5  The FDA’s Final Rule banning powdered surgeon’...   

                                    Requested Action  \
0  The petitioner asks the FDA Commissioner to in...   
1  FDA is requested to determine that Levetiracet...   
2  Macleods requests FDA to designate the approve...   
4  AHPA requests that FDA:\n1. Withdraw the propo...   
5  MREPC requests a two-month extension of the Fi...   

                           Justification for Request  
0  The petition outlines that the U.S. cryobank i...  
1  - The proposed product matches the RLD in conc...  
2  - The current RLD (Pyrazinamide 500 mg by Dava...  
4  - The current reference (Herbs of Commerce, 1s...  
5  - The short notice between the Final Rule anno...

In [51]:
# merge first
long_df = pd.merge(
    petitions_2017_main,
    responses_2017_mod,
    on = "File ID",
    how = "left"
)

# keep rows where the response happened after the petition was filed
invalid_responses_df = long_df[pd.to_datetime(long_df["Date of Response"], errors="coerce") <= pd.to_datetime(long_df["Date of Petition"], errors="coerce")
]
valid_responses_df = long_df[pd.to_datetime(long_df["Date of Response"], errors="coerce") > pd.to_datetime(long_df["Date of Petition"], errors="coerce")
]

#display(invalid_responses_df)
# petitions w/ recon are ["3196", "3832", "6689", "6692"]
recon_petit_id = petitions_2017_mod[petitions_2017_mod["File Name"].str.lower().str.contains("reconsider", regex=False)]
filter_recon = valid_responses_df[valid_responses_df["File ID"].isin(recon_petit_id["File ID"])]

filter_recon.to_csv("filter_reconsiderations_merged.csv", index=False)
display(filter_recon)

File ID                                 Petition File Name  \
107  fda-2017-p-3196  FDA-2017-P-3196-0001_Citizen_Petition_from_Phi...   
108  fda-2017-p-3196  FDA-2017-P-3196-0001_Citizen_Petition_from_Phi...   
110  fda-2017-p-3196  FDA-2017-P-3196-0072_Citizen_Petition_for_Reco...   
133  fda-2017-p-3832  FDA-2017-P-3832-0001_Citizen_Petition_for_Reco...   
134  fda-2017-p-3832  FDA-2017-P-3832-0001_Citizen_Petition_for_Reco...   
239  fda-2017-p-6689  FDA-2017-P-6689-0001_Citizen_Petition_from_Nov...   
240  fda-2017-p-6689  FDA-2017-P-6689-0001_Citizen_Petition_from_Nov...   
241  fda-2017-p-6689  FDA-2017-P-6689-0001_Citizen_Petition_from_Nov...   
242  fda-2017-p-6689  FDA-2017-P-6689-0001_Citizen_Petition_from_Nov...   
245  fda-2017-p-6689  FDA-2017-P-6689-0007_Petition_for_Reconsiderat...   
246  fda-2017-p-6689  FDA-2017-P-6689-0007_Petition_for_Reconsiderat...   
247  fda-2017-p-6692  FDA-2017-P-6692-0001_Citizen_Petition_from_Dru...   
248  fda-2017-p-6692  FDA-2017-P-6692-0001_Citizen_Petition_from_Dru...   
249  fda-2017-p-6692  FDA-2017-P-6692-0001_Citizen_Petition_from_Dru...   
252  fda-2017-p-6692  FDA-2017-P-6692-0043_Citizen_Petition_for_Reco...   

    Date of Petition  Was_Withdrawn      Document Type  \
107        5/18/2017          False  original petition   
108        5/18/2017          False  original petition   
110       11/13/2017          False    reconsideration   
133        6/16/2017          False    reconsideration   
134        6/16/2017          False    reconsideration   
239       11/29/2017          False  original petition   
240       11/29/2017          False  original petition   
241       11/29/2017          False  original petition   
242       11/29/2017          False  original petition   
245         6/7/2019          False    reconsideration   
246         6/7/2019          False    reconsideration   
247       11/21/2017          False  original petition   
248       11/21/2017          False  original petition   
249       11/21/2017          False  original petition   
252        7/31/2018          False    reconsideration   

                                             Submitter        Submitter Type  \
107  Individual citizen petition submitted jointly ...            individual   
108  Individual citizen petition submitted jointly ...            individual   
110  Philip Spiller, along with:  George A. Gellert...            individual   
133                    Porzio, Bromberg & Newman, P.C.        law/consulting   
134                    Porzio, Bromberg & Newman, P.C.        law/consulting   
239                                Novitium Pharma LLC  industry/corporation   
240                                Novitium Pharma LLC  industry/corporation   
241                                Novitium Pharma LLC  industry/corporation   
242                                Novitium Pharma LLC  industry/corporation   
245                          Arnall Golden Gregory LLP        law/consulting   
246                          Arnall Golden Gregory LLP        law/consulting   
247                     Drug Watch International, Inc.  industry/corporation   
248                     Drug Watch International, Inc.  industry/corporation   
249                     Drug Watch International, Inc.  industry/corporation   
252                     Drug Watch International, Inc.  industry/corporation   

                Petition Cited Statutes or Regulations  \
107  • 21 C.F.R. § 10.20 & § 10.30 (FDA citizen‑pet...   
108  • 21 C.F.R. § 10.20 & § 10.30 (FDA citizen‑pet...   
110  - 21 C.F.R. § 10.33 (Request for reconsiderati...   
133  - 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- A...   
134  - 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- A...   
239  - 21 C.F.R. § 10.25(a)\n- 21 C.F.R. § 10.30\n-...   
240  - 21 C.F.R. § 10.25(a)\n- 21 C.F.R. § 10.30\n-...   
241  - 21 C.F.R. § 10.25(a)\n- 21 C.F.R. § 10.30\n-...   
242  - 21 C.F.R. § 10.25(a)\n- 21 C.F.R. § 10.30\n-...   
245  - 21 C.F.R. § 10.

during the merging stage, an original petition date can get paired up with either the response date for the og petition (good; what we want) OR the response date for the reconsideration (not good) (after filtering i.e look at df above)

the fix: look at specific subset of df (orig vs recon) and within each subset, compare the different dates of response (bc the date of response for recon is bigger than the date of resp. for orig.)

In [ ]:
def find_correct_pairs(group):
    # (parameter) group: subset dataframe of long_df (merged); this subset df is long_df grouped by file id and date of petition -> 
    # get a subset of original petit and reconsideration
    petition_date = group["Date of Petition"].iloc[0]
    valid_resp = group[pd.to_datetime(group["Date of Response"], errors="coerce") > pd.to_datetime(petition_date, errors="coerce")]

    # Within the valid responses, find the earliest interim
    interims = valid_resp[valid_resp['Response to Petition'].str.lower().str.contains("interim", na=False)]
    earliest_interim_date = interims["Date of Response"].min() 

    # within the valid responses, find the latest FINAL decision (non-interim)
    finals = valid_resp[~valid_resp["Response to Petition"].str.lower().str.contains("interim", na=False)]
    latest_final_date = finals["Date of Response"].max() 
    
    # get the final decision w/ that (above ^) latest final date
    final_decision_text = finals.loc[finals["Date of Response"] == latest_final_date, "Response to Petition"].iloc[0] if not finals.empty else None

    # return a series w/ the results for the specific group
    return pd.Series({
        "Interim Response Date": earliest_interim_date,
        "Final Decision": final_decision_text,
        "Last Response Date": latest_final_date
    })

aggregated_responses = long_df.groupby(["File ID", "Date of Petition"]).apply(find_correct_pairs).reset_index()
final_df = pd.merge(
    petitions_2017_main,
    aggregated_responses,
    on=['File ID', 'Date of Petition'],
    how='left'
)

display(aggregated_responses)
display(final_df[final_df["File ID"].isin(recon_petit_id["File ID"])])


/var/folders/w1/yj3_r24s4tl7vlj_flvb3ntm0000gn/T/ipykernel_11637/2310531430.py:25: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_responses = long_df.groupby(["File ID", "Date of Petition"]).apply(find_correct_pairs).reset_index()


File ID Date of Petition Interim Response Date Final Decision  \
0    fda-2017-p-0052         1/1/2017                   NaN            NaN   
1    fda-2017-p-0115       12/22/2016                   NaN      Withdrawn   
2    fda-2017-p-0137         1/9/2017              7/7/2017           None   
3    fda-2017-p-0176         1/9/2017              2/2/2018           None   
4    fda-2017-p-0290        1/17/2017                   NaN         Denied   
..               ...              ...                   ...            ...   
190  fda-2017-p-6968       12/21/2017                   NaN       Approved   
191  fda-2017-p-7031       12/22/2017                   NaN            NaN   
192  fda-2017-p-7032       12/13/2017                   NaN       Approved   
193  fda-2017-p-7033       12/13/2017                   NaN       Approved   
194  fda-2017-p-7034       12/19/2017                   NaN            NaN   

    Last Response Date  
0                  NaN  
1            6/12/2023  
2                  NaN  
3                  NaN  
4            6/26/2017  
..                 ...  
190          3/14/2018  
191                NaN  
192          8/12/2021  
193          4/18/2023  
194                NaN  

[195 rows x 5 columns]

File ID                                 Petition File Name  \
78   fda-2017-p-3196  FDA-2017-P-3196-0001_Citizen_Petition_from_Phi...   
79   fda-2017-p-3196  FDA-2017-P-3196-0072_Citizen_Petition_for_Reco...   
95   fda-2017-p-3832  FDA-2017-P-3832-0001_Citizen_Petition_for_Reco...   
180  fda-2017-p-6689  FDA-2017-P-6689-0001_Citizen_Petition_from_Nov...   
181  fda-2017-p-6689  FDA-2017-P-6689-0007_Petition_for_Reconsiderat...   
182  fda-2017-p-6692  FDA-2017-P-6692-0001_Citizen_Petition_from_Dru...   
183  fda-2017-p-6692  FDA-2017-P-6692-0043_Citizen_Petition_for_Reco...   

    Date of Petition  Was_Withdrawn      Document Type  \
78         5/18/2017          False  original petition   
79        11/13/2017          False    reconsideration   
95         6/16/2017          False    reconsideration   
180       11/29/2017          False  original petition   
181         6/7/2019          False    reconsideration   
182       11/21/2017          False  original petition   
183        7/31/2018          False    reconsideration   

                                             Submitter        Submitter Type  \
78   Individual citizen petition submitted jointly ...            individual   
79   Philip Spiller, along with:  George A. Gellert...            individual   
95                     Porzio, Bromberg & Newman, P.C.        law/consulting   
180                                Novitium Pharma LLC  industry/corporation   
181                          Arnall Golden Gregory LLP        law/consulting   
182                     Drug Watch International, Inc.  industry/corporation   
183                     Drug Watch International, Inc.  industry/corporation   

                Petition Cited Statutes or Regulations  \
78   • 21 C.F.R. § 10.20 & § 10.30 (FDA citizen‑pet...   
79   - 21 C.F.R. § 10.33 (Request for reconsiderati...   
95   - 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- A...   
180  - 21 C.F.R. § 10.25(a)\n- 21 C.F.R. § 10.30\n-...   
181  - 21 C.F.R. § 10.25\n- 21 C.F.R. § 10.33\n- 21...   
182  - 21 C.F.R. §§ 10.20, 10.25, 10.30 (general pe...   
183  - 21 C.F.R. § 10.33 (Administrative reconsider...   

                               FDA Action Commented On  \
78   The fish‑consumption advice jointly issued by ...   
79   FDA’s denial (on October 13, 2017) of a May 18...   
95   The Final Rule: Food Labeling: Revision of the...   
180  FDA’s designation of Reference Standards for g...   
181  FDA’s decision to designate ANDA 074190 (West-...   
182  The petition comments on the lack of inclusion...   
183  The FDA’s July 2, 2018 denial of Drug Watch In...   

                                      Requested Action  \
78   1. Withdraw the 18 Jan 2017 advice immediately...   
79   - Reconsider and reverse the denial of the ori...   
95   - Stay the effective date of the Final Rule (J...   
180  The petitioner requests that the FDA designate...   
181  1. Reconsider the FDA’s decision to designate ...   
182  The petitioner requests the FDA to issue a Fin...   
183  The petitioner requests the FDA to reconsider ...   

                             Justification for Request Interim Response Date  \
78   The petitioners argue that the current advice ...                   NaN   
79   - FDA's 2017 advice misrepresents scientific e...                   NaN   
95   - Implementation of the rule is inconsistent w...            12/13/2017   
180  - The current RLD/RS, NAPROSYN Oral Suspension...             5/23/2018   
181  - Naprosyn (RLD and former RS) is currently av...                   NaN   
182  - Marijuana and THC are Schedule I substances ...             5/24/2018   
183  The justification includes multiple arguments:...                   NaN   

                            Final Decision Last Response Date  
78                                  Denied         10/13/2017  
79                                  Denied         10/10/2018  
95   Partially Approved / Partially Denied          6/13/2018  
180           

In [ ]:
# # final decision for each petition (ignoring interim)
# final_decisions = responses_2017_mod[~responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Response to Petition"].first()
# #final_decisions

# # count of interim responses for each file id
# interim_counts = responses_2017_mod[responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID").size()
# #interim_counts

# # date of interim & very last response for each file id
# final_decision_dates = responses_2017_mod[~responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Date of Response"].max()
# interim_dates = responses_2017_mod[responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Date of Response"].first()

# # responding FDA center
# fda_center = responses_2017_mod.groupby("File ID")["Responding FDA Center"].first()

# # cited statutes or regulations (ignoring interim)
# statutes_regs = responses_2017_mod[~responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Cited Statutes or Regulations"].first()

# # justification for interim response 
# just_interim = responses_2017_mod[responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Justification for Response"].first()

# # justification for final response
# just_final = responses_2017_mod[~responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Justification for Response"].first()

# # new responses df
# responses_agg = pd.DataFrame({
#     "Final Decision": final_decisions,
#     "Number of Interim Responses": interim_counts,
#     "Interim Response Date": interim_dates,
#     "Last Response Date": final_decision_dates,
#     "Responding FDA Center": fda_center,
#     "Responses Cited Statutes or Regulations": statutes_regs,
#     "Justification for Interim Response": just_interim,
#     "Justification for Final Response": just_final
# }).reset_index()

# # fill in 0 for petitions that had a final decision but no interim responses
# responses_agg["Number of Interim Responses"] = responses_agg["Number of Interim Responses"].fillna(0)
# #responses_agg["Interim_Response_Date"] = responses_agg["Interim_Response_Date"].fillna(np.NaN)

# # merging into FINAL DF!
# final_df = pd.merge(petitions_2017_main, responses_agg, on = "File ID", how = "left")

# display(responses_agg.head(3))
# display(final_df.head(3))

File ID Final Decision  Number of Interim Responses  \
0  fda-2017-p-0052         Denied                          1.0   
1  fda-2017-p-0115      Withdrawn                          0.0   
2  fda-2017-p-0137            NaN                          1.0   

  Interim Response Date Last Response Date  \
0                  None                NaN   
1                   NaN          6/12/2023   
2              7/7/2017                NaN   

                               Responding FDA Center  \
0  Center for Biologics Evaluation and Research (...   
1    Center for Drug Evaluation and Research (CDER)    
2    Center for Drug Evaluation and Research (CDER)    

             Responses Cited Statutes or Regulations  \
0  - Section 361 of the Public Health Service Act...   
1                                   Not applicable.    
2                                                NaN   

                  Justification for Interim Response  \
0  The FDA cites "existence of other FDA prioriti...   
1                                                NaN   
2  The FDA stated that the petition "raises compl...   

                    Justification for Final Response  
0  The FDA denied the petition based on the follo...  
1  - FDA contacted the petitioner (Jubilant Gener...  
2                                                NaN

File ID Date of Petition  Was_Withdrawn      Document Type  \
0  fda-2017-p-0052         1/1/2017          False  original petition   
1  fda-2017-p-0115       12/22/2016          False  original petition   
2  fda-2017-p-0137         1/9/2017           True  original petition   

                          Submitter        Submitter Type  \
0            Donor Sibling Registry     advocacy/academic   
1         Jubilant Generics Limited  industry/corporation   
2  Macleods Pharmaceuticals Limited  industry/corporation   

              Petition Cited Statutes or Regulations  \
0  - 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...   
1  - Section 505(j)(2)(C) of the FD&C Act [21 U.S...   
2  - Section 505(j) of the Federal Food, Drug, an...   

                             FDA Action Commented On  \
0  Lack of regulation and oversight in the sperm ...   
1  FDA’s requirement to approve an ANDA petition ...   
2  The petition comments on the absence of a curr...   

                                    Requested Action  \
0  The petitioner asks the FDA Commissioner to in...   
1  FDA is requested to determine that Levetiracet...   
2  Macleods requests FDA to designate the approve...   

                           Justification for Request Final Decision  \
0  The petition outlines that the U.S. cryobank i...         Denied   
1  - The proposed product matches the RLD in conc...      Withdrawn   
2  - The current RLD (Pyrazinamide 500 mg by Dava...            NaN   

   Number of Interim Responses Interim Response Date Last Response Date  \
0                          1.0                  None                NaN   
1                          0.0                   NaN          6/12/2023   
2                          1.0              7/7/2017                NaN   

                               Responding FDA Center  \
0  Center for Biologics Evaluation and Research (...   
1    Center for Drug Evaluation and Research (CDER)    
2    Center for Drug Evaluation and Research (CDER)    

             Responses Cited Statutes or Regulations  \
0  - Section 361 of the Public Health Service Act...   
1                                   Not applicable.    
2                                                NaN   

                  Justification for Interim Response  \
0  The FDA cites "existence of other FDA prioriti...   
1                                                NaN   
2  The FDA stated that the petition "raises compl...   

                    Justification for Final Response  
0  The FDA denied the petition based on the follo...  
1  - FDA contacted the petitioner (Jubilant Gener...  
2                                                NaN

# OCT 15, 2025 - More Cleaning / Moving Code from Visualizations File

### clean up "Final Decision" column, add column for year
for "clean up "Final Decision" column"
* https://docs.google.com/document/d/1r4FFpNyBWMhDrEHhMZ73AWrU0YSY2Eh895rBR_usk5o/edit?tab=t.0
* the data collection notes says common responses are "Interim Response, Approved, Denied, Dismissed (moot), 
* Partially Approved / Partially Denied, Other (Request for Additional Information)" but my data has "withdrawn"
* worried aobut consistency (though have not looked at other datasets w/ different years)

In [20]:
# clean up "Final Decision" column
def clean_decision(decision):
    if pd.isna(decision):
        return "no decision"
    
    decision_string_lower = str(decision).lower() 

    # "partially denied" vs "Partially Approved / Partially Denied"
    if "partially approved" in decision_string_lower and "denied" in decision_string_lower:
        return "partially approved / denied"
    
    # "approved" vs "approved (Determination Issued)"
    if "approved" in decision_string_lower:
        return "approved"
        
    # other more straightforward decisions
    if "denied" in decision_string_lower:
        return "denied"
    if "dismissed" in decision_string_lower:
        return "dismissed (moot)"
    if "withdrawn" in decision_string_lower:
        return "withdrawn"
        
    return "other"


# make new column for cleaned final decision  
final_df["Cleaned Final Decision"] = final_df["Final Decision"].apply(clean_decision)

In [21]:
# add year column
# pat = r'fda-(\d+)'
# t = final_df["File ID"].str.extract(pat).iloc[0]
# t.unique()

def add_year_col(dataset):
    pat = r'fda-(\d+)'
    years_series = final_df["File ID"].str.extract(pat).iloc[:, 0]
    dataset["Year"] = years_series
        
add_year_col(final_df)
display(final_df.head())

File ID Date of Petition  Was_Withdrawn  \
0  fda-2017-p-0052         1/1/2017          False   
1  fda-2017-p-0115       12/22/2016          False   
2  fda-2017-p-0137         1/9/2017           True   
3  fda-2017-p-0176         1/9/2017          False   
4  fda-2017-p-0290        1/17/2017          False   

                                           Submitter        Submitter Type  \
0                             Donor Sibling Registry     advocacy/academic   
1                          Jubilant Generics Limited  industry/corporation   
2                   Macleods Pharmaceuticals Limited  industry/corporation   
3                  Kleinfeld, Kaplan and Becker, LLP        law/consulting   
4  Malaysian Rubber Export Promotion Council (MREPC)     advocacy/academic   

              Petition Cited Statutes or Regulations  \
0  - 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...   
1  - Section 505(j)(2)(C) of the FD&C Act [21 U.S...   
2  - Section 505(j) of the Federal Food, Drug, an...   
3  - 21 C.F.R. § 10.30 (Citizen Petition procedur...   
4  - FDA Docket No. FDA-2015-N-5017 (Final Rule)\...   

                             FDA Action Commented On  \
0  Lack of regulation and oversight in the sperm ...   
1  FDA’s requirement to approve an ANDA petition ...   
2  The petition comments on the absence of a curr...   
3  The FDA’s proposed amendment to 21 C.F.R. § 10...   
4  The FDA’s Final Rule banning powdered surgeon’...   

                                    Requested Action  \
0  The petitioner asks the FDA Commissioner to in...   
1  FDA is requested to determine that Levetiracet...   
2  Macleods requests FDA to designate the approve...   
3  AHPA requests that FDA:\n1. Withdraw the propo...   
4  MREPC requests a two-month extension of the Fi...   

                           Justification for Request Final Decision  \
0  The petition outlines that the U.S. cryobank i...         Denied   
1  - The proposed product matches the RLD in conc...      Withdrawn   
2  - The current RLD (Pyrazinamide 500 mg by Dava...            NaN   
3  - The current reference (Herbs of Commerce, 1s...            NaN   
4  - The short notice between the Final Rule anno...         Denied   

   Number of Interim Responses Interim Response Date Last Response Date  \
0                          1.0                  None                NaN   
1                          0.0                   NaN          6/12/2023   
2                          1.0              7/7/2017                NaN   
3                          1.0              2/2/2018                NaN   
4                          0.0                   NaN          6/26/2017   

                               Responding FDA Center  \
0  Center for Biologics Evaluation and Research (...   
1    Center for Drug Evaluation and Research (CDER)    
2    Center for Drug Evaluation and Research (CDER)    
3  Center for Food Safety and Applied Nutrition (...   
4  Center for Devices and Radiological Health (CD...   

             Responses Cited Statutes or Regulations  \
0  - Section 361 of the Public Health Service Act...   
1                                   Not applicable.    
2                                                NaN   
3                                                NaN   
4  - 21 CFR 1.94, 21 CFR 1.95, 21 CFR 1.96\n- 21 ...   

                  Justification for Interim Response  \
0  The FDA cites "existence of other FDA prioriti...   
1                                                NaN   
2  The FDA stated that the petition "raises compl...   
3  The FDA stated the petition is under active ev...   
4                                                NaN   

                    Justification for Final Response Cleaned Final Decision  \
0  The FDA denied the petition based on the follo...                 denied   
1  - FDA contacted the petitioner (Jubilant Gener...              withdrawn   
2                                                NaN            no decision   
3

### Calculate "Response Time (Days)" 
* calculating how many days from when the petition was submitted till when it got a final decision (not just interim)
*  "Response Time (Days)" is a censored datatype bc there are petitions that never got a final decision from the FDA (look at cell below)
*  preparation for survival analysis
*  petitions WITH an interim but NO final decision, petitions w/ NO final decision & NO interim, petitions w/ MISSING response date (0052) will have their "Last Response Date" imputed with "2025-10-15", the "current" date (even if they got an interim because we only care about the response time of the FDA's FINAL decision)


**(FIXED OCT 15) QUESTION/PROBLEM 1 (above):**

so when calculating the response time for petitions w/o final decision (and were not withdrawn), if the petition had an interim file inputted, it will use the date of interim as the "Last Response Date" in df["Response Time (Days)"] = (df["Last Response Date"] - df["Date of Petition"]).dt.days 

(the problem) there exists petitions that have no final decision and no interim (so no calculation for resp time like last 2 rows of df above 1391, 1463). looking at the spa files, i found out that these two have an acknowledgement and nothing else which _im guessing acknowledgement is not the same as interim which then means that the fda never even got back with an interim._ 

this type of data is known as **censored data** because it is still ongoing/stuck waiting for a response.

In [24]:
# filter petitions that wasn't withdrawn by petitioner
exclude_withdrawals_petit = final_df[final_df["Was_Withdrawn"] == False]

# petitions that were not withdrawn and never got a final decision (but some got interims)
# check with spa files to make sure there is actually no response file

# (exlcude_withdrawals_petit["Submitter Type"] == "industry/corporation")
petit_no_decision = exclude_withdrawals_petit[(exclude_withdrawals_petit["Cleaned Final Decision"] == "no decision")]
petit_no_decision.to_csv("no_decision_petitions.csv", index=False)

display(petit_no_decision.head(3))
len(petit_no_decision) # 27 petitions w/ no decision

File ID Date of Petition  Was_Withdrawn  \
3   fda-2017-p-0176       2017-01-09          False   
9   fda-2017-p-0508       2017-01-25          False   
34  fda-2017-p-1298       2017-03-02          False   

                                            Submitter     Submitter Type  \
3                   Kleinfeld, Kaplan and Becker, LLP     law/consulting   
9   Union of Concerned Scientists, Center for Scie...  advocacy/academic   
34                      The Good Food Institute (GFI)  advocacy/academic   

               Petition Cited Statutes or Regulations  \
3   - 21 C.F.R. § 10.30 (Citizen Petition procedur...   
9   - 5 U.S.C. § 553(e)\n- 21 C.F.R. § 10.25\n- 21...   
34  - 21 C.F.R. § 10.30 (Citizen petition)\n- 5 U....   

                              FDA Action Commented On  \
3   The FDA’s proposed amendment to 21 C.F.R. § 10...   
9   - Current FDA regulations allowing health and ...   
34  GFI comments on recent suggestions that plant-...   

                                     Requested Action  \
3   AHPA requests that FDA:\n1. Withdraw the propo...   
9   - Amend 21 C.F.R. §§ 101.13 and 101.14 to incl...   
34  GFI requests that FDA:\n1. Issue a regulation ...   

                            Justification for Request Final Decision  \
3   - The current reference (Herbs of Commerce, 1s...            NaN   
9   - Scientific evidence links excessive added su...            NaN   
34  - Terms like “soy milk” and “vegan cheese” hav...            NaN   

    Number of Interim Responses Interim Response Date Last Response Date  \
3                           1.0              2/2/2018         2025-10-15   
9                           1.0             7/17/2017         2025-10-15   
34                          1.0             8/29/2017         2025-10-15   

                                Responding FDA Center  \
3   Center for Food Safety and Applied Nutrition (...   
9   Center for Food Safety and Applied Nutrition (...   
34  Center for Food Safety and Applied Nutrition (...   

   Responses Cited Statutes or Regulations  \
3                                      NaN   
9                                      NaN   
34                                     NaN   

                   Justification for Interim Response  \
3   The FDA stated the petition is under active ev...   
9   The FDA stated that it has not completed revie...   
34  - The petition requested FDA to issue regulati...   

   Justification for Final Response Cleaned Final Decision  Year  \
3                               NaN            no decision  2017   
9                               NaN            no decision  2017   
34                              NaN            no decision  2017   

    Response Time (Days)  
3                 3201.0  
9                 3185.0  
34                3149.0

27

In [23]:
# calculate response time
def calc_resp_time(dataset):
    dataset["Last Response Date"] = dataset["Last Response Date"].fillna(pd.to_datetime("2025-10-15"))
    dataset["Date of Petition"] = pd.to_datetime(dataset["Date of Petition"], errors="coerce")
    dataset["Last Response Date"] = pd.to_datetime(dataset["Last Response Date"], errors="coerce")
    dataset["Response Time (Days)"] = (dataset["Last Response Date"] - dataset["Date of Petition"]).dt.days

calc_resp_time(final_df)
display(final_df.head())
final_df.to_csv("final_version_petitions_and_responses.csv", index=False)

File ID Date of Petition  Was_Withdrawn  \
0  fda-2017-p-0052       2017-01-01          False   
1  fda-2017-p-0115       2016-12-22          False   
2  fda-2017-p-0137       2017-01-09           True   
3  fda-2017-p-0176       2017-01-09          False   
4  fda-2017-p-0290       2017-01-17          False   

                                           Submitter        Submitter Type  \
0                             Donor Sibling Registry     advocacy/academic   
1                          Jubilant Generics Limited  industry/corporation   
2                   Macleods Pharmaceuticals Limited  industry/corporation   
3                  Kleinfeld, Kaplan and Becker, LLP        law/consulting   
4  Malaysian Rubber Export Promotion Council (MREPC)     advocacy/academic   

              Petition Cited Statutes or Regulations  \
0  - 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...   
1  - Section 505(j)(2)(C) of the FD&C Act [21 U.S...   
2  - Section 505(j) of the Federal Food, Drug, an...   
3  - 21 C.F.R. § 10.30 (Citizen Petition procedur...   
4  - FDA Docket No. FDA-2015-N-5017 (Final Rule)\...   

                             FDA Action Commented On  \
0  Lack of regulation and oversight in the sperm ...   
1  FDA’s requirement to approve an ANDA petition ...   
2  The petition comments on the absence of a curr...   
3  The FDA’s proposed amendment to 21 C.F.R. § 10...   
4  The FDA’s Final Rule banning powdered surgeon’...   

                                    Requested Action  \
0  The petitioner asks the FDA Commissioner to in...   
1  FDA is requested to determine that Levetiracet...   
2  Macleods requests FDA to designate the approve...   
3  AHPA requests that FDA:\n1. Withdraw the propo...   
4  MREPC requests a two-month extension of the Fi...   

                           Justification for Request Final Decision  \
0  The petition outlines that the U.S. cryobank i...         Denied   
1  - The proposed product matches the RLD in conc...      Withdrawn   
2  - The current RLD (Pyrazinamide 500 mg by Dava...            NaN   
3  - The current reference (Herbs of Commerce, 1s...            NaN   
4  - The short notice between the Final Rule anno...         Denied   

   Number of Interim Responses Interim Response Date Last Response Date  \
0                          1.0                  None         2025-10-15   
1                          0.0                   NaN         2023-06-12   
2                          1.0              7/7/2017         2025-10-15   
3                          1.0              2/2/2018         2025-10-15   
4                          0.0                   NaN         2017-06-26   

                               Responding FDA Center  \
0  Center for Biologics Evaluation and Research (...   
1    Center for Drug Evaluation and Research (CDER)    
2    Center for Drug Evaluation and Research (CDER)    
3  Center for Food Safety and Applied Nutrition (...   
4  Center for Devices and Radiological Health (CD...   

             Responses Cited Statutes or Regulations  \
0  - Section 361 of the Public Health Service Act...   
1                                   Not applicable.    
2                                                NaN   
3                                                NaN   
4  - 21 CFR 1.94, 21 CFR 1.95, 21 CFR 1.96\n- 21 ...   

                  Justification for Interim Response  \
0  The FDA cites "existence of other FDA prioriti...   
1                                                NaN   
2  The FDA stated that the petition "raises compl...   
3  The FDA stated the petition is under active ev...   
4                                                NaN   

                    Justification for Final Response Cleaned Final Decision  \
0  The FDA denied the petition based on the follo...                 denied   
1  - FDA contacted the petitioner (Jubilant Gener...              withdrawn   
2                                                NaN            no decision   
3

#### fixing prob #2 in visualizations file 
* peititons that have the original petititon and a reconsideration petition will have an extra row with slightly tweaked "File ID" (i.e a unique file id for orig and recon)

In [40]:
# petitions w/ reconsideration petition (3196, 3832, 6689, 6692)
recon_petit_id = petitions_2017_mod[petitions_2017_mod["File Name"].str.lower().str.contains("reconsider", regex=False)]
#["File ID"]
display(petitions_2017_main[petitions_2017_main["File ID"].str.lower().isin(recon_petit_id["File ID"])])
display(recon_petit_id)

File ID Date of Petition  Was_Withdrawn  \
92   fda-2017-p-3196        5/18/2017          False   
93   fda-2017-p-3196       11/13/2017          False   
111  fda-2017-p-3832        6/16/2017          False   
214  fda-2017-p-6689       11/29/2017          False   
215  fda-2017-p-6689         6/7/2019          False   
216  fda-2017-p-6692       11/21/2017          False   
217  fda-2017-p-6692        7/31/2018          False   

                                             Submitter        Submitter Type  \
92   Individual citizen petition submitted jointly ...            individual   
93   Philip Spiller, along with:  George A. Gellert...            individual   
111                    Porzio, Bromberg & Newman, P.C.        law/consulting   
214                                Novitium Pharma LLC  industry/corporation   
215                          Arnall Golden Gregory LLP        law/consulting   
216                     Drug Watch International, Inc.  industry/corporation   
217                     Drug Watch International, Inc.  industry/corporation   

                Petition Cited Statutes or Regulations  \
92   • 21 C.F.R. § 10.20 & § 10.30 (FDA citizen‑pet...   
93   - 21 C.F.R. § 10.33 (Request for reconsiderati...   
111  - 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- A...   
214  - 21 C.F.R. § 10.25(a)\n- 21 C.F.R. § 10.30\n-...   
215  - 21 C.F.R. § 10.25\n- 21 C.F.R. § 10.33\n- 21...   
216  - 21 C.F.R. §§ 10.20, 10.25, 10.30 (general pe...   
217  - 21 C.F.R. § 10.33 (Administrative reconsider...   

                               FDA Action Commented On  \
92   The fish‑consumption advice jointly issued by ...   
93   FDA’s denial (on October 13, 2017) of a May 18...   
111  The Final Rule: Food Labeling: Revision of the...   
214  FDA’s designation of Reference Standards for g...   
215  FDA’s decision to designate ANDA 074190 (West-...   
216  The petition comments on the lack of inclusion...   
217  The FDA’s July 2, 2018 denial of Drug Watch In...   

                                      Requested Action  \
92   1. Withdraw the 18 Jan 2017 advice immediately...   
93   - Reconsider and reverse the denial of the ori...   
111  - Stay the effective date of the Final Rule (J...   
214  The petitioner requests that the FDA designate...   
215  1. Reconsider the FDA’s decision to designate ...   
216  The petitioner requests the FDA to issue a Fin...   
217  The petitioner requests the FDA to reconsider ...   

                             Justification for Request  
92   The petitioners argue that the current advice ...  
93   - FDA's 2017 advice misrepresents scientific e...  
111  - Implementation of the rule is inconsistent w...  
214  - The current RLD/RS, NAPROSYN Oral Suspension...  
215  - Naprosyn (RLD and former RS) is currently av...  
216  - Marijuana and THC are Schedule I substances ...  
217  The justification includes multiple arguments:...

File Name Date of Petition  \
93   FDA-2017-P-3196-0072_Citizen_Petition_for_Reco...       11/13/2017   
111  FDA-2017-P-3832-0001_Citizen_Petition_for_Reco...        6/16/2017   
215  FDA-2017-P-6689-0007_Petition_for_Reconsiderat...         6/7/2019   
217  FDA-2017-P-6692-0043_Citizen_Petition_for_Reco...        7/31/2018   

    Date Comments                      Identity of Submitting Entity  \
93            NaN  Philip Spiller, along with:\n- George A. Gelle...   
111           NaN                    Porzio, Bromberg & Newman, P.C.   
215           NaN                          Arnall Golden Gregory LLP   
217           NaN                     Drug Watch International, Inc.   

                                Representation Details  \
93                                      Not mentioned.   
111  Representing Natural Products Association (NPA...   
215  Representing Atnahs Pharma US, Ltd. and Athena...   
217                                     Not mentioned.   

                         Cited Statutes or Regulations  \
93   - 21 C.F.R. § 10.33 (Request for reconsiderati...   
111  - 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- A...   
215  - 21 C.F.R. § 10.25\n- 21 C.F.R. § 10.33\n- 21...   
217  - 21 C.F.R. § 10.33 (Administrative reconsider...   

                               FDA Action Commented On  \
93   FDA’s denial (on October 13, 2017) of a May 18...   
111  The Final Rule: Food Labeling: Revision of the...   
215  FDA’s decision to designate ANDA 074190 (West-...   
217  The FDA’s July 2, 2018 denial of Drug Watch In...   

                                      Requested Action  \
93   - Reconsider and reverse the denial of the ori...   
111  - Stay the effective date of the Final Rule (J...   
215  1. Reconsider the FDA’s decision to designate ...   
217  The petitioner requests the FDA to reconsider ...   

                             Justification for Request  Status of Review  \
93   - FDA's 2017 advice misrepresents scientific e...             False   
111  - Implementation of the rule is inconsistent w...             False   
215  - Naprosyn (RLD and former RS) is currently av...             False   
217  The justification includes multiple arguments:...             False   

                                       Review Comments  \
93                                                 NaN   
111                                                NaN   
215  Different representation status from original ...   
217                                                NaN   

                                             Submitter        Submitter Type  \
93   Philip Spiller, along with:  George A. Gellert...            individual   
111                    Porzio, Bromberg & Newman, P.C.        law/consulting   
215                          Arnall Golden Gregory LLP        law/consulting   
217                     Drug Watch International, Inc.  industry/corporation   

             File ID  Withdrawal File  Was_Withdrawn  
93   fda-2017-p-3196            False          False  
111  fda-2017-p-3832            False          False  
215  fda-2017-p-6689            False          False  
217  fda-2017-p-6692            False          False

In [36]:
responses_2017_mod[responses_2017_mod["File ID"].str.lower().isin(recon_petit_id["File ID"])]

File Name Date of Response  \
94   FDA-2017-P-3196-0071_Response_Letter_from_CFSA...       10/13/2017   
95   FDA-2017-P-3196-0074_Reconsideration_Denial_CP...       10/10/2018   
114  FDA-2017-P-3832-0003_Interim_Response_from_FDA...       12/13/2017   
115  FDA-2017-P-3832-0004_FDA_2017_P_3832___NPA_Res...        6/13/2018   
205  FDA-2017-P-6689-0004_Interim_response_from_FDA...        5/23/2018   
206  FDA-2017-P-6689-0006_Letter_from_FDA_CDER_to_N...         5/8/2019   
207  FDA-2017-P-6689-0010_Letter_from_FDA_CDER_to_A...        7/12/2019   
208  FDA-2017-P-6689-0013_Petition_Response_Letter_...        5/26/2020   
209  FDA-2017-P-6692-0040_Interim_Response_from_FDA...        5/24/2018   
210  FDA-2017-P-6692-0042_Petition_Denial_Letter_fr...         7/2/2018   
211  FDA-2017-P-6692-0102_Letter_from_FDA_CDER_to_D...        7/18/2019   

    Date Comments                              Responding FDA Center  \
94            NaN  Center for Food Safety and Applied Nutrition (...   
95            NaN  Not explicitly named, but signed by Leslie Kux...   
114           NaN  Center for Food Safety and Applied Nutrition (...   
115           NaN  Center for Food Safety and Applied Nutrition (...   
205           NaN     Center for Drug Evaluation and Research (CDER)   
206           NaN     Center for Drug Evaluation and Research (CDER)   
207           NaN     Center for Drug Evaluation and Research (CDER)   
208           NaN     Center for Drug Evaluation and Research (CDER)   
209           NaN     Center for Drug Evaluation and Research (CDER)   
210           NaN     Center for Drug Evaluation and Research (CDER)   
211           NaN     Center for Drug Evaluation and Research (CDER)   

                      Response to Petition  \
94                                  Denied   
95                                  Denied   
114                       Interim Response   
115  Partially Approved / Partially Denied   
205                       Interim Response   
206                               Approved   
207                                 Denied   
208                                 Denied   
209                       Interim Response   
210                                 Denied   
211                                 Denied   

                         Cited Statutes or Regulations  \
94   - 21 CFR § 10.30(e)(3) – Governing denial of c...   
95   - 21 CFR § 10.33(d) – Standard for granting a ...   
114  - 21 CFR 10.30(e)(2) – Allows for interim resp...   
115  - 21 CFR § 101.9(c)(6)(i) – Definition of diet...   
205                                 21 CFR 10.30(e)(2)   
206  - 21 U.S.C. § 355(j) (Section 505(j) of the FD...   
207  - 21 CFR 10.35(e) (criteria for administrative...   
208  - 21 CFR 10.33 (Petition for Reconsideration)\...   
209                                 21 CFR 10.30(e)(2)   
210  - 21 CFR part 310 (Negative Monographs)\n- 21 ...   
211  - 21 CFR 10.33(d) and 10.33(e) (Petitions for ...   

                            Justification for Response          File ID  
94   - The 2017 seafood advisory was developed thro...  fda-2017-p-3196  
95   - The petitioner failed to show that relevant ...  fda-2017-p-3196  
114  - The FDA indicates that due to other agency p...  fda-2017-p-3832  
115  - FDA reaffirmed that defining "dietary fiber"...  fda-2017-p-3832  
205  FDA has not yet reached a decision due to othe...  fda-2017-p-6689  
206  - The originally designated reference standard...  fda-2017-p-6689  
207  - The petition failed to meet 3 of the 4 crite...  fda-2017-p-6689  
208  - The petitioner failed to meet the regulatory...  fda-2017-p-6689  
209  FDA was unable to reach a decision due to comp...  fda-2017-p-6692  
210  - The petition requested that marijuana and TH...  fda-2017-p-6692  
211  - The Reconsideration Petition included new in...  fda-2017-p-6692